In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from scikeras.wrappers import KerasRegressor


from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import make_scorer, get_scorer

In [5]:
df = pd.read_csv(r"..\oblig3\abalone.data", sep=",", names=["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Rings"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole weight    4177 non-null   float64
 5   Shucked weight  4177 non-null   float64
 6   Viscera weight  4177 non-null   float64
 7   Shell weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [25]:
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
num_features = df.drop(columns=['Rings', 'Sex'], axis=1)
num_features = pd.DataFrame(StandardScaler().fit_transform(num_features), columns=num_features.columns)
features = pd.concat([num_features, df['Sex']], axis=1)
targets = df['Rings']

0       15
1        7
2        9
3       10
4        7
        ..
4172    11
4173    10
4174     9
4175    10
4176    12
Name: Rings, Length: 4177, dtype: int64

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,2
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,2
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,0
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,2
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,1
...,...,...,...,...,...,...,...,...
4172,0.341509,0.424464,0.609334,0.118813,0.047908,0.532900,0.073062,0
4173,0.549706,0.323686,-0.107991,0.279929,0.358808,0.309362,0.155685,2
4174,0.632985,0.676409,1.565767,0.708212,0.748559,0.975413,0.496955,2
4175,0.841182,0.777187,0.250672,0.541998,0.773341,0.733627,0.410739,0


In [44]:
global_random_state = 15

scoring = {
    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
    'r2': get_scorer('r2'),
}  

def evaluate(estimator, X, y):
    scores = {}
    for (name,scorer) in scoring.items():
        scores[name] = scorer(estimator, X, y) 
    return scores

def train(features, targets, estimator, params, scoring=scoring, refit='r2', random_state=global_random_state, outer_splits=5, inner_splits=4):

    outer_cv = StratifiedKFold(n_splits=outer_splits, shuffle=True, random_state=global_random_state)
    inner_cv = StratifiedKFold(n_splits=inner_splits, shuffle=True, random_state=global_random_state)

    scores_train = []
    scores_test = []
    estimators = []
    cv_results = []

    # Loop through all test folds
    for (train_index, test_index) in outer_cv.split(features, targets):
        
        grid = GridSearchCV(
            estimator,
            params, 
            scoring=scoring, 
            refit=refit,
            error_score='raise', 
            cv=inner_cv)
        grid.fit(features.iloc[train_index], targets.iloc[train_index])        
        
        evaluation_train = evaluate(grid, features.iloc[train_index], targets.iloc[train_index])
        evaluation_test = evaluate(grid, features.iloc[test_index], targets.iloc[test_index])
        
        scores_train.append(evaluation_train)
        scores_test.append(evaluation_test)
        
        estimators.append(grid.best_estimator_)
        cv_results.append(pd.DataFrame(grid.cv_results_))
        print("*")

    return estimators, pd.DataFrame(scores_train), pd.DataFrame(scores_test), pd.concat(cv_results, names=['test_split'], keys=range(outer_splits))

def create_model(optimizer="adam", kernel_initializer='glorot_uniform'):
    model = Sequential()
    model.add(Input(shape=(8,)))
    model.add(Dense(30, activation='relu', kernel_initializer=kernel_initializer))
    model.add(Dense(10, activation='relu', kernel_initializer=kernel_initializer))
    model.add(Dense(1, kernel_initializer=kernel_initializer))
    model.compile(optimizer=optimizer, loss = 'mean_squared_error', metrics=['mean_absolute_error'])
    return model

In [48]:
param_grid = {
    #'batch_size': [10, 20, 40],
    #'epochs': [10, 20, 40],
    #'optimizer': ['SGD', 'Adam', 'RMSprop'],
    'model__kernel_initializer': ['glorot_uniform', 'normal', 'uniform'],     
    'model__optimizer': ['adam', 'sgd']
    #'model__kernel_initializer': ['glorot_uniform', 'he_normal']
}

model = KerasRegressor(model=create_model, verbose=1)

keras_reg_estimators, keras_reg_scores_train, keras_reg_scores_test, keras_reg_cv_results = train(features, targets, model, param_grid)


c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83.9464 - mean_absolute_error: 8.4995 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 72.2076 - mean_absolute_error: 7.7713  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 83.4808 - mean_absolute_error: 8.3408  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 88.3249 - mean_absolute_error: 8.4607  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.2421 - mean_absolute_error: 2.4287 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.3268 - mean_absolute_error: 2.3663 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.0611 - mean_absolute_error: 2.4956   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16.8410 - mean_absolute_error: 2.7738   
27/27 ━━━━━━━━━━━━━━━━━━━

c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 86.5708 - mean_absolute_error: 8.7703 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 85.0222 - mean_absolute_error: 8.6050    
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 100.4903 - mean_absolute_error: 9.4765 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 93.2965 - mean_absolute_error: 9.0462   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.2151 - mean_absolute_error: 2.4806 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.2115 - mean_absolute_error: 2.3655  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.3173 - mean_absolute_error: 2.6538  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.8586 - mean_absolute_error: 2.5212   
27/27 ━━━━━━━━━━━━━━━━

c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 84.4000 - mean_absolute_error: 8.3632   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 74.3489 - mean_absolute_error: 7.9624  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 70.0798 - mean_absolute_error: 7.5951  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 84.5967 - mean_absolute_error: 8.4435  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.2868 - mean_absolute_error: 2.6047   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.0209 - mean_absolute_error: 2.4608  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 16.1669 - mean_absolute_error: 2.6226   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12.8611 - mean_absolute_error: 2.4779  
27/27 ━━━━━━━━━━━━━━━

c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 85.3742 - mean_absolute_error: 8.4857    
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 105.9917 - mean_absolute_error: 9.7140  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 96.8799 - mean_absolute_error: 9.1901  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 81.6244 - mean_absolute_error: 8.4613    
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.6494 - mean_absolute_error: 2.6615   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.5277 - mean_absolute_error: 2.6244 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14.4908 - mean_absolute_error: 2.6050  
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 18.8558 - mean_absolute_error: 2.9157   
27/27 ━━━━━━━━━━━━

c:\Users\hallo\Documents\GitHub\praktisk-maskinlering\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 84.5340 - mean_absolute_error: 8.5654    
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 66.8063 - mean_absolute_error: 7.3737 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 77.1153 - mean_absolute_error: 8.1246 
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 99.4122 - mean_absolute_error: 9.3880    
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.3545 - mean_absolute_error: 2.5133   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 13.2095 - mean_absolute_error: 2.4964   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.6474 - mean_absolute_error: 2.3510   
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.5949 - mean_absolute_error: 2.6815  
27/27 ━━━━━━━━━━━━━

In [49]:
keras_reg_scores_test

,mean_absolute_error,mean_squared_error,r2
0,-2.494768,-12.308616,-0.146867
1,-1.690500,-5.190036,0.496165
2,-1.742375,-5.096104,0.503501
3,-1.715742,-5.769391,0.446065
4,-1.717323,-6.521204,0.363775
